<a href="https://colab.research.google.com/github/bhattacharjee/mtu-nlp-assignment/blob/main/assignment2/notebooks/Augmentation_wip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install tqdm
!pip install torchvision==0.8.2  -q -q -q
!pip install torchtext==0.8.1    -q -q -q
!pip install torchaudio==0.7.2   -q -q -q
!pip install torch==1.7.1        -q -q -q
!pip install tqdm==4.49.0        -q -q -q
!pip install transformers==4.2.2 -q -q -q
!pip install tensorflow          -q -q -q
!pip install tensorboard         -q -q -q
!pip install tensorboardX        -q -q -q
!pip install --upgrade virtualenv -q -q -q
!pip install sentencepiece==0.1.94       -q -q -q
!pip install mosestokenizer==1.1.0 -q -q -q

!rm -rf mtu-nlp-assignment
!git clone https://github.com/bhattacharjee/mtu-nlp-assignment.git
!cp mtu-nlp-assignment/work/augment_data/convert_to_squad.py .
!cp mtu-nlp-assignment/work/augment_data/util.py .
!cp mtu-nlp-assignment/work/augment_data/args.py .

from google.colab import drive
import shutil
drive.mount('/content/gdrive')

Cloning into 'mtu-nlp-assignment'...
remote: Enumerating objects: 1027, done.
remote: Counting objects: 100% (1027/1027), done.
remote: Compressing objects: 100% (892/892), done.
remote: Total 1027 (delta 508), reused 383 (delta 131), pack-reused 0
Receiving objects: 100% (1027/1027), 96.10 MiB | 34.04 MiB/s, done.
Resolving deltas: 100% (508/508), done.
Mounted at /content/gdrive


In [11]:
%%writefile copy_train.sh
#!/bin/bash
cp /content/gdrive/MyDrive/NLP/NLP-Save-Train/train.tar.gz /tmp
cd /tmp
gunzip train.tar.gz
tar -xf train.tar
ls /tmp/train

Writing copy_train.sh


In [12]:
!/bin/bash copy_train.sh

attention_mask	id	   offset_mapping	       start_positions
end_positions	input_ids  overflow_to_sample_mapping


In [20]:
!unzip mtu-nlp-assignment/assignment2/robustqa.zip
!mv robustqa/datasets_50k.tar.gz datasets_50k.tar
!tar -xf datasets_50k.tar

Archive:  mtu-nlp-assignment/assignment2/robustqa.zip
replace robustqa/datasets_50k.tar.gz? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: robustqa/datasets_50k.tar.gz  
  inflating: __MACOSX/robustqa/._datasets_50k.tar.gz  
  inflating: robustqa/convert_to_squad.py  
  inflating: robustqa/environment.yml  
  inflating: robustqa/util.py        
  inflating: robustqa/README.md      
  inflating: __MACOSX/robustqa/._README.md  
  inflating: robustqa/train.py       
  inflating: robustqa/args.py        


# BackTranslation Implementation

In [21]:
from transformers import MarianMTModel, MarianTokenizer
import torch

if torch.cuda.is_available():
    print("Using cuda")
    torch_device = 'cuda'
    cuda = torch.device('cuda')
    print(cuda)
    #torch.set_default_tensor_type('torch.cuda.FloatTensor')
else:
    torch_device = 'cpu'

target_model_name = 'Helsinki-NLP/opus-mt-en-ROMANCE'
target_tokenizer = MarianTokenizer.from_pretrained(target_model_name)
target_model = MarianMTModel.from_pretrained(target_model_name)

en_model_name = 'Helsinki-NLP/opus-mt-ROMANCE-en'
en_tokenizer = MarianTokenizer.from_pretrained(en_model_name)
en_model = MarianMTModel.from_pretrained(en_model_name)

if torch.cuda.is_available():
    target_model = target_model.to(torch_device)
    en_model = en_model.to(torch_device)

def translate(textarray, model, tokenizer, lang="fr"):
    # Prepare the text data into appropriate format for the model
    def templatize(text):
        if lang== 'en':
            return text
        else:
            return f">>{lang}<< {text}"
    textarray = [templatize(t) for t in textarray]
    encoded = tokenizer.prepare_seq2seq_batch(textarray, return_tensors='pt').to(torch_device)
    translated = model.generate(**encoded).to(torch_device)
    translated_texts = tokenizer.batch_decode(translated, skip_special_tokens=True)
    return translated_texts

def back_translate(texts, source_lang="en", target_lang="fr"):
    # Translate from source to target language
    if isinstance(texts, str):
        texts = [texts]
    fr_texts = translate(texts, target_model, target_tokenizer, 
                         lang=target_lang)
    # Translate from target language back to source language
    back_translated_texts = translate(fr_texts, en_model, en_tokenizer, 
                                      lang=source_lang)
    return back_translated_texts

Using cuda
cuda


# Read squad and translate

In [22]:
#/usr/bin/env python3
import json
import os
from collections import OrderedDict
import torch
import csv
import util
from transformers import DistilBertTokenizerFast
from transformers import DistilBertForQuestionAnswering
from transformers import AdamW
from args import *


from torch.utils.data import DataLoader
from torch.utils.data.sampler import RandomSampler, SequentialSampler
from args import get_train_test_args

from tqdm import tqdm
import random


def get_dataset(args, datasets, data_dir, tokenizer, split_name):
    datasets = datasets.split(',')
    dataset_dict = None
    dataset_name=''
    for dataset in datasets:
        dataset_name += f'_{dataset}'
        dataset_dict_curr = util.read_squad(f'{data_dir}/{dataset}')
        dataset_dict = util.merge(dataset_dict, dataset_dict_curr)
    return dataset_dict

def print_question(question, context, ind, answer):
    print(f"ID:            {ind}")
    print(f"CONTEXT:       {context}")
    print(f"QUESTOIN:      {context}")
    print(f"ANSWER:        {answer}")
    anslen = len(ans)

def get_new_context(context:str, answer:str)->tuple:
    """
    Replace the answer in the context with a number
    so that it doesn't backtranslate

    Return a tuple:
        1. string that was replaced, handy while restoring
           the answer in the translated string
        2. translated question
    """
    replaced_str = ""
    start_ind = context.index(answer)
    orig_ans = answer
    len_orig_ans = len(orig_ans)
    context = list(context)

    for i in range(len_orig_ans):
        replaced_str +=  '1'
        context[i + start_ind] = '1'

    print('-' * 80)
    print(replaced_str)
    print('-' * 80)
    return "".join(context), replaced_str

def get_start_end_index(context, replaced):
    with open("save.txt", "a") as f:
        print(context, file=f)
        print(replaced, file=f)

    try:
        start_ind = context.index(replaced)
    except:
        # Sometimes the replaced string is truncated by the
        # translation, try again with half the string to see
        # if there is a match
        if len(replaced) < 4:
            return -1, -1
        try:
            start_ind = context.index(replaced[:len(replaced) // 2])
        except:
            if len(replaced) < 8:
                return -1, -1
            try:
                start_ind = context.index(replaced[len(replaced) // 4])
            except:
                return -1, -1

    start_char = replaced[0]
    end_ind = start_ind
    while end_ind < len(context) and context[end_ind] == start_char:
        end_ind += 1
    return start_ind, end_ind


    
def reconstruct_context(context:str, replaced:str, origans:str)->str:
    """
    Take the backtranslated context, and replace the placeholder
    for the answer with the actual answer
    """
    start_index, end_index = get_start_end_index(context, replaced)
    str1 = context[:start_index]
    str2 = context[end_index:]
    return str1 + origans + str2

def back_translate_context(context, answer):
    temp_ctx, replaced = get_new_context(context, answer)
    trns_ctx = back_translate(temp_ctx)[0]
    new_ctx = reconstruct_context(trns_ctx, replaced, answer)
    return new_ctx


def main():
    # define parser and arguments
    import sys
    sys.argv = ['']
    args = get_train_test_args()

    util.set_seed(args.seed)
    print(args)

    if not os.path.exists(args.save_dir):
        os.makedirs(args.save_dir)
    args.save_dir = util.get_save_dir(args.save_dir, args.run_name)
    log = util.get_logger(args.save_dir, 'log_train')
    train_dataset = get_dataset(args, args.train_datasets, args.train_dir, None, 'train')

    print(train_dataset.keys())

    forig = open("orig.txt", "w")
    fnew = open("new.txt", "w")

    questions = train_dataset['question']
    contexts = train_dataset['context']
    ids = train_dataset['id']
    answers = train_dataset['answer']
    j = 0
    for q, c, i, a in zip(questions, contexts, ids, answers):
        if j == 4:
            answer = a['text'][0]
            nc = back_translate_context(c, answer)
            nq = back_translate(q)

            print(c, file=forig)
            print(q, file=forig)
            print(answer, file=forig)
            print("", file=forig)

            print(nc, file=fnew)
            print(nq, file=fnew)
            print(answer, file=fnew)
            print("", file=fnew)

        j += 1
        if j > 10: break

    forig.close()
    fnew.close()


main()

Namespace(batch_size=16, do_eval=False, do_train=False, eval=False, eval_datasets='race,relation_extraction,duorc', eval_dir='datasets/oodomain_test', eval_every=5000, lr=3e-05, num_epochs=3, num_visuals=10, recompute_features=False, run_name='multitask_distilbert', save_dir='save/', seed=42, sub_file='', train=False, train_datasets='squad,nat_questions,newsqa', train_dir='datasets/indomain_train', val_dir='datasets/indomain_val', visualize_predictions=False)
dict_keys(['question', 'context', 'id', 'answer'])
--------------------------------------------------------------------------------
111111111
--------------------------------------------------------------------------------


In [5]:
%tb

SystemExit: ignored